In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.abspath("../src/"))
import model.util as model_util
import model.profile_models as profile_models
import model.binary_models as binary_models
import extract.data_loading as data_loading
import extract.compute_predictions as compute_predictions
import extract.compute_shap as compute_shap
import plot.viz_sequence as viz_sequence
import pyBigWig
import torch
import numpy as np
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import scipy.stats
import json
import tqdm
import h5py
tqdm.tqdm_notebook()  # It is necessary to call this before the tqdm.notebook submodule is available

In [ ]:
# Plotting defaults
font_manager.fontManager.ttflist.extend(
    font_manager.createFontList(
        font_manager.findSystemFonts(fontpaths="/users/amtseng/modules/fonts")
    )
)
plot_params = {
    "figure.titlesize": 22,
    "axes.titlesize": 22,
    "axes.labelsize": 20,
    "legend.fontsize": 18,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "font.family": "Roboto",
    "font.weight": "bold"
}
plt.rcParams.update(plot_params)

### Define paths for the model and data of interest

In [ ]:
model_type = "binary"

In [ ]:
# Shared paths/constants
raw_data_base_path = "/users/amtseng/att_priors/data/raw/"
proc_data_base_path = "/users/amtseng/att_priors/data/processed/"
model_base_path = "/users/amtseng/att_priors/models/trained_models/%s/" % model_type
tfm_results_path = "/users/amtseng/att_priors/results/tfmodisco/%s/" % model_type
chrom_set = ["chr1"]
input_length = 1346 if model_type == "profile" else 1000
profile_length = 1000
reference_fasta = "/users/amtseng/genomes/mm10.fasta"
chrom_sizes = "/users/amtseng/genomes/mm10.canon.chrom.sizes"

In [ ]:
# BPNet
condition_name = "BPNet"
files_spec_path = os.path.join(proc_data_base_path, "BPNet_ChIPseq/%s/config/BPNet_training_paths.json" % model_type)
num_tasks = 3
num_strands = 2
if model_type == "profile":
    controls = "shared"
    model_class = profile_models.ProfilePredictorWithSharedControls
else:
    model_class = binary_models.BinaryPredictor

bpnet_tasks = ["Nanog", "Oct4", "Sox2"]
task_index = None

if task_index is None:
    footprints_paths = [
        os.path.join(raw_data_base_path, "BPNet_ChIPnexus/BPNet_%s_peaks-idr.bed.gz" % task.upper())
        for task in bpnet_tasks
    ]
    pos_bigwigs = [
        os.path.join(raw_data_base_path, "BPNet_ChIPnexus/BPNet_%s_pos.bw" % task.upper())
        for task in bpnet_tasks
    ]
    neg_bigwigs = [
        os.path.join(raw_data_base_path, "BPNet_ChIPnexus/BPNet_%s_neg.bw" % task.upper())
        for task in bpnet_tasks
    ]
else:
    footprints_paths = [os.path.join(raw_data_base_path, "BPNet_ChIPnexus/BPNet_%s_peaks-idr.bed.gz" % bpnet_tasks[task_index].upper())]
    pos_bigwigs = [os.path.join(raw_data_base_path, "BPNet_ChIPnexus/BPNet_%s_pos.bw" % bpnet_tasks[task_index].upper())]
    neg_bigwigs = [os.path.join(raw_data_base_path, "BPNet_ChIPnexus/BPNet_%s_neg.bw" % bpnet_tasks[task_index].upper())]
# noprior_model_path = os.path.join(model_base_path, "BPNet/20/model_ckpt_epoch_18.pt")
# prior_model_path = os.path.join(model_base_path, "BPNet_prior/25/model_ckpt_epoch_17.pt")
# noprior_tfm_path = os.path.join(tfm_results_path, "BPNet/BPNet_r20_e18_task%s_tfm.h5" % ("agg" if task_index is None else str(task_index)))
# prior_tfm_path = os.path.join(tfm_results_path, "BPNet/BPNet_prior_r25_e17_task%s_tfm.h5" % ("agg" if task_index is None else str(task_index)))
noprior_model_path = os.path.join(model_base_path, "BPNet/22/model_ckpt_epoch_1.pt")
prior_model_path = os.path.join(model_base_path, "BPNet_prior/27/model_ckpt_epoch_11.pt")
noprior_tfm_path = os.path.join(tfm_results_path, "BPNet/BPNet_r22_e1_task%s_tfm.h5" % ("agg" if task_index is None else str(task_index)))
prior_tfm_path = os.path.join(tfm_results_path, "BPNet/BPNet_prior_r27_e11_task%s_tfm.h5" % ("agg" if task_index is None else str(task_index)))

In [ ]:
torch.set_grad_enabled(True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def restore_model(model_path):
    model = model_util.restore_model(model_class, model_path)
    model.eval()
    model = model.to(device)
    return model

In [ ]:
# Import the model without priors
noprior_model = restore_model(noprior_model_path)

In [ ]:
# Import the model with priors
prior_model = restore_model(prior_model_path)

### Import footprits
For BPNet, use the ChIP-nexus peaks, which can act as an easy proxy for footprints, althought they are very wide footprints

In [ ]:
footprints = [
    pd.read_csv(
        path, sep="\t", header=None,
        names=[
            "chrom", "peak_start", "peak_end", "name", "score", "strand",
            "signal", "pval", "qval", "summit_offset"
        ]
    )
    for path in footprints_paths
]
footprints = pd.concat(footprints)
footprints = footprints[footprints["chrom"].isin(chrom_set)]

### Create reader for BigWigs
Only for something like BPNet, which has ChIP-nexus profiles, which offer a "continuous" view of footprint confidence, rather than a simple binary footprint classification

In [ ]:
def get_profiles(chrom, start, end):
    """
    Returns N-array of BigWig values, as sum of negative and positive strands.
    """
    result = np.zeros((end - start, 2))
    for i in range(len(neg_bigwigs)):
        with pyBigWig.open(neg_bigwigs[i], "r") as f:
            result[:, 0] = f.values(chrom, start, end)
        with pyBigWig.open(pos_bigwigs[i], "r") as f:
            result[:, 1] = f.values(chrom, start, end)
    return np.sum(np.nan_to_num(result), axis=1)

### Data preparation
Create an input data loader, that maps coordinates or bin indices to data needed for the model

In [ ]:
if model_type == "profile":
    input_func = data_loading.get_profile_input_func(
        files_spec_path, input_length, profile_length, reference_fasta
    )
    pos_set = data_loading.get_positive_profile_coords(
        files_spec_path, task_ind=task_index, chrom_set=chrom_set
    )
else:
    input_func = data_loading.get_binary_input_func(
        files_spec_path, input_length, reference_fasta
    )
    pos_set = data_loading.get_positive_binary_bins(
        files_spec_path, task_ind=task_index, chrom_set=chrom_set
    )

### Compute importances

In [ ]:
def compute_gradients(model, sample):
    """
    Given an array of N coordinates or bins, computes the input gradients
    for the model, returning an N x I x 4 array of gradient values and an
    N x I x 4 array of one-hot encoded sequence.
    """
    if model_type == "profile":
        results = compute_predictions.get_profile_model_predictions(                                              
            model, sample, num_tasks, input_func, controls=controls,                        
            return_losses=False, return_gradients=True, show_progress=True                                         
        )
    else:
        results = compute_predictions.get_binary_model_predictions(                                              
            model, sample, input_func,                      
            return_losses=False, return_gradients=True, show_progress=True                                         
        )
    return results["input_grads"], results["input_seqs"]

In [ ]:
def compute_shap_scores(model, sample, batch_size=128):
    """
    Given an array of N coordinates or bins, computes the SHAP scores
    for the model, returning an N x I x 4 array of SHAP scores and an
    N x I x 4 array of one-hot encoded sequence.
    """
    num_samples = len(sample)
    num_batches = int(np.ceil(num_samples / batch_size))
    
    all_shap_scores = np.empty((num_samples, input_length, 4))
    all_one_hot_seqs = np.empty((num_samples, input_length, 4))
        
    if model_type == "profile":
        shap_explainer = compute_shap.create_profile_explainer(
            model, input_length, profile_length, num_tasks, num_strands, controls,
            task_index=task_index
        )
    else:
        shap_explainer = compute_shap.create_binary_explainer(
            model, input_length, task_index=task_index
        )

    for i in tqdm.notebook.trange(num_batches):
        batch_slice = slice(i * batch_size, (i + 1) * batch_size)
        batch = sample[batch_slice]

        if model_type == "profile":
            input_seqs, profiles = input_func(batch)
            shap_scores = shap_explainer(
                input_seqs, cont_profs=profiles[:, num_tasks:], hide_shap_output=True
            )
        else:
            input_seqs, _, _ = input_func(batch)
            shap_scores = shap_explainer(
                input_seqs, hide_shap_output=True
            )

        all_shap_scores[batch_slice] = shap_scores
        all_one_hot_seqs[batch_slice] = input_seqs
    return all_shap_scores, all_one_hot_seqs

In [ ]:
num_samples = 1000
rng = np.random.RandomState(20200318)
sample = pos_set[rng.choice(len(pos_set), size=num_samples, replace=False)]

In [ ]:
# Get coordinates for the sample
if model_type == "profile":
    # For profile models, add a random jitter to avoid center-bias
    jitters = np.random.randint(-128, 128 + 1, size=len(sample))
    sample[:, 1] = sample[:, 1] + jitters
    sample[:, 2] = sample[:, 2] + jitters
    sample_coords = sample
else:
    sample_coords = input_func(sample)[2]

In [ ]:
# Compute the importance scores and 1-hot seqs
imp_type = ("DeepSHAP scores", "input gradients")[0]
imp_func = compute_shap_scores if imp_type == "DeepSHAP scores" else compute_gradients
noprior_imp_scores, sample_input_seqs = imp_func(
    noprior_model, sample
)
prior_imp_scores, _ = imp_func(
    prior_model, sample
)

In [ ]:
# Pad the coordinates on both sides symmetrically to make them `input_length` 
centers = (sample_coords[:, 1] + sample_coords[:, 2]) // 2
starts = centers - (input_length // 2)
ends = starts + input_length
sample_coords[:, 1] = starts
sample_coords[:, 2] = ends

In [ ]:
# Plot out a few examples
for i in np.random.choice(num_samples, size=3, replace=False):
    print(i, sample[i])
    print("=========================")
    print("Without priors:")
    plt.figure(figsize=(20, 2))
    plt.plot(np.sum(noprior_imp_scores[i] * sample_input_seqs[i], axis=1))
    plt.show()
    print("With priors:")
    plt.figure(figsize=(20, 2))
    plt.plot(np.sum(prior_imp_scores[i] * sample_input_seqs[i], axis=1))
    plt.show()

### Quantify agreement of importances with footprints using overlap
In each importance score track (over the entire input region), do the importances tend to be elevated more in known footprints?

In [ ]:
def compute_footprint_overlap_amount(coords, score_tracks, footprints):
    """
    From an N x 3 object array of coordinates, a parallel N x I x 4 array
    of score tracks, and a table of imported footprints, this function computes
    and returns the overlap amount for each of the N examples, where the overlap
    for an example is the fraction of score magnitudes in its track that fall
    within a footprint. An overlap will be NaN if there are no footprints in the
    coordinate. Returns an N-array.
    """
    score_tracks = np.abs(score_tracks)  # Take absolute value of score tracks
    overlaps = np.empty(len(coords))
    for i in tqdm.notebook.trange(len(coords)):
        chrom, start, end = coords[i]
        # Filter down the set of footprints to only those that overlap with the
        # coordinate
        fps = footprints[
            (footprints["chrom"] == chrom) & (footprints["peak_start"] <= end) & \
            (start <= footprints["peak_end"])
        ]
        mask = np.zeros(end - start, dtype=bool)
        intervals = fps[["peak_start", "peak_end"]].values - start
        for interval in intervals:
            mask[interval[0]:interval[1]] = True
        if not np.sum(mask):
            overlaps[i] = np.nan
        else:
            overlaps[i] = np.sum(score_tracks[i][mask]) / np.sum(score_tracks[i])
    return overlaps

In [ ]:
noprior_imp_fp_overlaps = compute_footprint_overlap_amount(
    sample_coords, noprior_imp_scores * sample_input_seqs, footprints
)
prior_imp_fp_overlaps = compute_footprint_overlap_amount(
    sample_coords, prior_imp_scores * sample_input_seqs, footprints
)

# Remove the NaNs, which is where no footprint was in the coordinate
noprior_imp_fp_overlaps = noprior_imp_fp_overlaps[np.isfinite(noprior_imp_fp_overlaps)]
prior_imp_fp_overlaps = prior_imp_fp_overlaps[np.isfinite(prior_imp_fp_overlaps)]

In [ ]:
bin_num = 30
all_vals = np.concatenate([noprior_imp_fp_overlaps, prior_imp_fp_overlaps])
bins = np.linspace(np.min(all_vals), np.max(all_vals), bin_num)
fig, ax = plt.subplots(figsize=(16, 8))
ax.hist(noprior_imp_fp_overlaps, bins=bins, color="coral", label="No prior", alpha=0.7)
ax.hist(prior_imp_fp_overlaps, bins=bins, color="slateblue", label="With Fourier prior", alpha=0.7)
plt.legend()
title = "Histogram of %s overlap with BPNet %s ChIP-nexus peaks" % (imp_type, "aggregated" if task_index is None else bpnet_tasks[task_index])
title += "\n%s %s models" % (condition_name, model_type)
title += "\nComputed on %d randomly drawn test peaks" % num_samples
plt.title(title)
plt.xlabel("Proportion of importance in footprints")

print("Average score without priors: %f" % np.nanmean(noprior_imp_fp_overlaps))
print("Average score with priors: %f" % np.nanmean(prior_imp_fp_overlaps))
print("Standard error without priors: %f" % scipy.stats.sem(noprior_imp_fp_overlaps))
print("Standard error with priors: %f" % scipy.stats.sem(prior_imp_fp_overlaps))
w, p = scipy.stats.wilcoxon(prior_imp_fp_overlaps, noprior_imp_fp_overlaps, alternative="greater")
print("One-sided Wilcoxon test: W = %f, p = %f" % (w, p))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(noprior_imp_fp_overlaps, prior_imp_fp_overlaps, color="mediumorchid", alpha=0.5)
title = "Pairwise comparison of %s overlap with BPNet %s ChIP-nexus peaks" % (imp_type, "aggregated" if task_index is None else bpnet_tasks[task_index])
title += "\n%s %s models" % (condition_name, model_type)
title += "\nComputed on %d randomly drawn test peaks" % num_samples
plt.title(title)
limits = [
    np.min([ax.get_xlim(), ax.get_ylim()]),
    np.max([ax.get_xlim(), ax.get_ylim()]),
]
ax.plot(limits, limits, "--", alpha=0.5, color="black")
ax.set_aspect("equal")
ax.set_xlim(limits)
ax.set_ylim(limits)
plt.xlabel("Proportion of importance in footprints without prior")
plt.ylabel("Proportion of importance in footprints with Fourier prior")

### Quantify agreement of importance windows with footprints using ranks
Rank all of the windows by importance; do the windows at the top tend to overlap with footprints?

In [ ]:
def compute_single_footprint_overlap_mask(chrom, start, end, footprints):
    """
    Given a coordinate, returns a boolean mask for that coordinate
    for which bases lie within a footprint
    """
    mask = np.zeros(end - start, dtype=bool)
    for peak_bed in peak_beds:
        rows = footprints[
            (footprints["chrom"] == chrom) & (footprints["peak_start"] <= end) & \
            (start <= footprints["peak_end"])
        ]
        intervals = rows[["peak_start", "peak_end"]].values - start
        for interval in intervals:
            mask[interval[0]:interval[1]] = True
    return mask

In [ ]:
def compute_single_footprint_overlap_mask(chrom, start, end, footprints):
    """
    Given a coordinate, returns a boolean mask for that coordinate
    for which bases lie within a footprint
    """
    mask = np.zeros(end - start, dtype=bool)
    rows = footprints[
        (footprints["chrom"] == chrom) & (footprints["peak_start"] <= end) & \
        (start <= footprints["peak_end"])
    ]
    intervals = rows[["peak_start", "peak_end"]].values - start
    for interval in intervals:
        mask[interval[0]:interval[1]] = True
    return mask

def compute_footprint_overlap_mask(coords, footprints):
    """
    Given a sample of N coordinates or bins, extracts a boolean mask
    denoting which locations in each sample overlap a footprint.
    Returns an N x I boolean array, where each subarray is a boolean mask
    for which bases in that coordinate lie within a footprint.
    """
    masks = []
    for coord in tqdm.notebook.tqdm(coords):
        # Pad to profile length
        masks.append(compute_single_footprint_overlap_mask(coord[0], coord[1], coord[2], footprints))
    return np.stack(masks, axis=0)

In [ ]:
def compute_windowed_footprint_overlap_mask(scores, one_hot_seqs, seq_footprint_masks, window_size=10):
    """
    Windowing by `window_size`, computes the total importance magnitude for the
    actual importance in each window, and identifies which windows overlap a footprint.
    Returns a boolean NumPy array denoting which windows overlap a peak, sorted in
    descending order of window importance, and the indices of each window. This requires
    a mask of whether or not each base in each input sequence overlaps a footprint,
    computed by `compute_footprint_overlap_mask`.
    """
    num_samples = len(scores)
    all_window_imps, all_footprint_masks = [], []
    all_window_inds = []
    for i in tqdm.notebook.trange(num_samples):
        score_track = np.sum(np.abs(scores[i] * one_hot_seqs[i]), axis=1)  # Actual importance
        num_windows = len(score_track) - window_size + 1
        
        # Compute windowed peak mask
        window_locs = np.arange(num_windows) + ((len(score_track) - num_windows) / 2)
        windowed_footprint_mask = seq_footprint_masks[i][window_locs.astype(int)]
        
        # Compute sum of importance in each window
        importances = np.empty(num_windows)
        for j in range(num_windows):
            importances[j] = np.sum(score_track[j : j + window_size])
            all_window_inds.append([i, j])
        
        all_window_imps.append(importances)
        all_footprint_masks.append(windowed_footprint_mask)
    
    window_imps, all_footprint_masks = np.concatenate(all_window_imps), np.concatenate(all_footprint_masks)
    all_window_inds = np.stack(all_window_inds)
    return all_footprint_masks[np.flip(np.argsort(window_imps))], all_window_inds[np.flip(np.argsort(window_imps))]

In [ ]:
def normed_rank_enrichment(ordered_mask):
    """
    From a binary mask array (in order from best to worst thresholds), computes
    the rank enrichment at each threshold. Specifically, this computes a
    normalized CDF of how many 1s are seen in the first k instances.
    """
    cdf = np.cumsum(ordered_mask)
    # To normalize, divide by the expectation if there all 1s were spread
    # out evenly across the instances
    expectation = np.sum(ordered_mask) / len(ordered_mask) * np.arange(1, len(cdf) + 1)
    return cdf / expectation

In [ ]:
window_size = 1

In [ ]:
footprint_masks = compute_footprint_overlap_mask(sample_coords, footprints)
noprior_window_overlap_mask, noprior_windows = compute_windowed_footprint_overlap_mask(
    noprior_imp_scores, sample_input_seqs, footprint_masks, window_size=window_size
)
prior_window_overlap_mask, prior_windows = compute_windowed_footprint_overlap_mask(
    prior_imp_scores, sample_input_seqs, footprint_masks, window_size=window_size
)
noprior_window_overlap_cdf = normed_rank_enrichment(noprior_window_overlap_mask)
prior_window_overlap_cdf = normed_rank_enrichment(prior_window_overlap_mask)

Now let's see what was missed

In [ ]:
def show(indexes):
    i, j = indexes
    print(j)
    print(sample_coords[i])
    plt.figure(figsize=(20, 3))
    plt.plot(np.sum(prior_imp_scores[i] * sample_input_seqs[i], axis=1))
    plt.show()
    viz_sequence.plot_weights(prior_imp_scores[i][j - 10: j + 10])
    viz_sequence.plot_weights((prior_imp_scores[i] * sample_input_seqs[i])[j - 10: j + 10])
    
    plt.figure(figsize=(20, 3))
    plt.plot(np.sum(noprior_imp_scores[i] * sample_input_seqs[i], axis=1))
    plt.show()
    viz_sequence.plot_weights(noprior_imp_scores[i][j - 10: j + 10])
    viz_sequence.plot_weights((noprior_imp_scores[i] * sample_input_seqs[i])[j - 10: j + 10])

In [ ]:
inds = np.where(~prior_window_overlap_mask[:1000])[0]
print(inds)
# show(prior_windows[750])
# show(prior_windows[79])

In [ ]:
# Save all the missed sequences
# save_file = "/users/amtseng/att_priors/results/missed_seqlets/%s_%s_missed_seqlets.h5" % (condition_name, model_type)
# with h5py.File(save_file, "w") as f:
#     f.create_dataset("input_seqs", data=sample_input_seqs)
#     f.create_dataset("prior_hyp_scores", data=prior_imp_scores)
#     f.create_dataset("noprior_hyp_scores", data=noprior_imp_scores)
#     f.create_dataset("prior_ranked_bases", data=prior_windows)
#     f.create_dataset("noprior_ranked_bases", data=noprior_windows)
#     f.create_dataset("prior_missed_inds", data=np.where(~prior_window_overlap_mask)[0])
#     f.create_dataset("noprior_missed_inds", data=np.where(~noprior_window_overlap_mask)[0])

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
title = "Cumulative BPNet %s ChIP-nexus peak overlap over bases ranked by %s" % ("aggregated" if task_index is None else bpnet_tasks[task_index], imp_type)
title += "\n%s %s models" % (condition_name, model_type)
title += "\nComputed on %d randomly drawn test peaks" % num_samples
fig.suptitle(title)
ax[0].plot(np.cumsum(noprior_window_overlap_mask), label="No prior", color="coral")
ax[0].plot(np.cumsum(prior_window_overlap_mask), label="With Fourier prior", color="slateblue")
ax[0].set_ylabel("Number of bases in footprints (x1000)")
ax[0].set_yticklabels((ax[0].get_yticks() / 1000).astype(int))
ax[1].plot(noprior_window_overlap_cdf, label="No prior", color="coral")
ax[1].plot(prior_window_overlap_cdf, label="With Fourier prior", color="slateblue")
ax[1].set_ylabel("Enrichment of number of bases")
fig.text(0.45, 0.05, "Top k bases by importance", fontsize=18)
plt.legend()

In [ ]:
noprior_precis, noprior_recall, _ = sklearn.metrics.precision_recall_curve(
    noprior_window_overlap_mask.astype(int),
    np.flip(np.arange(len(noprior_window_overlap_mask))) / len(noprior_window_overlap_mask)
)
prior_precis, prior_recall, _ = sklearn.metrics.precision_recall_curve(
    prior_window_overlap_mask.astype(int),
    np.flip(np.arange(len(prior_window_overlap_mask))) / len(prior_window_overlap_mask)
)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
title = "Precision - Recall of BPNet %s ChIP-nexus peak overlap over bases ranked by %s" % ("aggregated" if task_index is None else bpnet_tasks[task_index], imp_type)
title += "\n%s %s models" % (condition_name, model_type)
title += "\nComputed on %d randomly drawn test peaks" % num_samples
plt.title(title)
plt.plot(noprior_recall[:-1], noprior_precis[:-1], color="coral", label="No prior")
plt.plot(prior_recall[:-1], prior_precis[:-1], color="slateblue", label="With Fourier prior")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()

print("auPRC without priors: %f" % sklearn.metrics.auc(noprior_recall[:-1], noprior_precis[:-1]))
print("auPRC with priors: %f" % sklearn.metrics.auc(prior_recall[:-1], prior_precis[:-1]))

### Quantify agreement of importance windows with profile-derived footprints using ranks
Rank all of the windows by importance; do the windows at the top tend to overlap with footprints? Instead of using pre-determined footprints, we use profile heights to binarize footprint classification

In [ ]:
def compute_max_profile_heights(chrom, start, end, profile_func, flank_size=35):
    """
    Given a coordinate, returns the maximum profile height within the given flank
    size (to the left and right), for each base. Returns an array of size equal to
    the size of the coordinate.
    """
    max_heights = np.empty(end - start)
    heights = profile_func(chrom, start - flank_size, end + flank_size)
    for i in range(end - start):
        max_heights[i] = np.max(heights[i : i + (flank_size * 2)])
    return max_heights

def compute_footprint_overlap_mask_by_profile(coords, profile_func, flank_size=35, quant_cutoff=0.9):
    """
    Given an N x 3 object array of coordinates, extracts a boolean mask
    denoting which locations in each sample overlap a footprint, based on
    profile height. For each coordinate, this will look `flank_size` base pairs
    left and right of each base pair and record the maximum profile height seen.
    A base is deemed to overlie a footprint if its maximum profile height is in the
    `quant_cutoff` percentile (default 90th percentile) over all given bases given.
    Returns an N x I boolean array, where each subarray is a boolean mask for which
    bases in that coordinate lie within a footprint based on the profile height.
    """
    max_heights = []
    for coord in tqdm.notebook.tqdm(coords):
        max_heights.append(compute_max_profile_heights(coord[0], coord[1], coord[2], profile_func, flank_size))
    max_heights = np.stack(max_heights, axis=0)
    cutoff = np.sort(np.ravel(max_heights))[int(quant_cutoff * max_heights.size)]
    return max_heights >= cutoff

In [ ]:
footprint_masks_by_profile = compute_footprint_overlap_mask_by_profile(sample_coords, get_profiles)
noprior_window_profoverlap_mask, noprior_windows = compute_windowed_footprint_overlap_mask(
    noprior_imp_scores, sample_input_seqs, footprint_masks_by_profile, window_size=window_size
)
prior_window_profoverlap_mask, prior_windows = compute_windowed_footprint_overlap_mask(
    prior_imp_scores, sample_input_seqs, footprint_masks_by_profile, window_size=window_size
)
noprior_window_profoverlap_cdf = normed_rank_enrichment(noprior_window_profoverlap_mask)
prior_window_profoverlap_cdf = normed_rank_enrichment(prior_window_profoverlap_mask)

In [ ]:
assert condition_name == "BPNet"
top_limit = 10000
fig, ax = plt.subplots(1, 3, figsize=(20, 6))
title = "Cumulative BPNet %s ChIP-nexus peak overlap by profile height in top windows of size %d by %s" % ("aggregated" if task_index is None else bpnet_tasks[task_index], window_size, imp_type)
title += "\n%s %s models" % (condition_name, model_type)
title += "\nOver %d random test peaks" % len(sample)
fig.suptitle(title)
ax[0].plot(np.cumsum(noprior_window_profoverlap_mask[:top_limit]), label="No prior", color="red")
ax[0].plot(np.cumsum(prior_window_profoverlap_mask[:top_limit]), label="With Fourier prior", color="blue")
ax[0].set_ylabel("Number of overlapping windows")
ax[1].plot(np.cumsum(noprior_window_profoverlap_mask), label="No prior", color="red")
ax[1].plot(np.cumsum(prior_window_profoverlap_mask), label="With Fourier prior", color="blue")
ax[1].set_ylabel("Number of overlapping windows")
ax[2].plot(noprior_window_profoverlap_cdf, label="No prior", color="red")
ax[2].plot(prior_window_profoverlap_cdf, label="With Fourier prior", color="blue")
ax[2].set_ylabel("Hypergeometric enrichment of number of overlapping windows")
fig.text(0.45, 0.05, "Top k windows of importance")
plt.legend()

### Import the TF-MoDISco results

In [ ]:
background_freqs = np.array([0.27, 0.23, 0.23, 0.27])
def pfm_info_content(pfm, pseudocount=0.001):
    """
    Given an L x 4 PFM, computes information content for each base and
    returns it as an L-array.
    """
    num_bases = pfm.shape[1]
    # Normalize track to probabilities along base axis
    pfm_norm = (pfm + pseudocount) / (np.sum(pfm, axis=1, keepdims=True) + (num_bases * pseudocount))
    ic = pfm_norm * np.log2(pfm_norm / np.expand_dims(background_freqs, axis=0))
    return np.sum(ic, axis=1)

def pfm_to_pwm(pfm, pseudocount=0.001):
    """
    Converts and L x 4 PFM into an L x 4 PWM.
    """
    num_bases = pfm.shape[1]
    # Incorporate pseudocount by adding it to every element and renormalizing
    pfm_norm = (pfm + pseudocount) / (np.sum(pfm, axis=1, keepdims=True) + (num_bases * pseudocount))
    return np.log2(pfm_norm / np.expand_dims(background_freqs, axis=0))

In [ ]:
def revcomp_motif(motif):
    return np.flip(motif, axis=(0, 1))

In [ ]:
def import_tfmodisco_motifs(
    tfm_results_hdf5, min_seqlets=750, min_ic=0.6, ic_window=6, trim_flank_ic_frac=0.2,
    plot_all_motifs=False, plot_passed_motifs=True
):
    """
    Imports the TF-MoDISco motifs, and a final set of motifs, trimmed by info content.
    The motifs returned must have at least `min_seqlets` supporting them, and there must
    be a window of size `ic_window` with at IC at least `min_ic`. Finally, the resulting
    motifs are trimmed by cutting off flanks whose base-level IC is below
    `trim_flank_ic_frac` of the highest IC of the motif. This also only keeps motifs with
    overall positive contributions (i.e. no negative seqlets).
    Returns 2 parallel lists: a list of motif CWMs, and a list of motif PWMs.
    """
    cwms, pwms = [], []
    num_seqlets = []
    with h5py.File(tfm_results_hdf5, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if plot_all_motifs:
                print("Metacluster: %s (%d/%d)" % (metacluster_key, metacluster_i + 1, num_metaclusters))
                print("==========================================")
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
                pattern_name = pattern_name.decode()
                pattern = patterns[pattern_name]
                seqlets = pattern["seqlets_and_alnmts"]["seqlets"]
                
                if plot_all_motifs:
                    print("Pattern: %s (%d/%d)" % (pattern_name, pattern_i + 1, num_patterns))
                    print("--------------------------------------")

                    print("%d seqlets" % len(seqlets))
                    print("Sequence")
                    viz_sequence.plot_weights(pattern["sequence"]["fwd"][:])
                    print("Hypothetical contributions")
                    viz_sequence.plot_weights(pattern["task0_hypothetical_contribs"]["fwd"][:])
                    print("Contribution_scores")
                    viz_sequence.plot_weights(pattern["task0_contrib_scores"]["fwd"][:])
                
                pfm = pattern["sequence"]["fwd"][:]
                act_contribs = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if np.sum(act_contribs) < 0:
                    continue
                
                # Check number of seqlets and IC
                if len(seqlets) < min_seqlets:
                    continue
                
                pwm = pfm_to_pwm(pfm)
                pwm_ic = pfm_info_content(pfm)
                max_windowed_ic = max(
                    np.sum(pwm_ic[i : (i + ic_window)]) for i in range(len(pwm_ic) - ic_window + 1)
                )
                if max_windowed_ic / ic_window < min_ic:
                    continue
                    
                # Cut off flanks from actual contribution scores and PWM based on IC of PWM
                ic_trim_thresh = np.max(pwm_ic) * trim_flank_ic_frac
                pass_inds = np.where(pwm_ic >= ic_trim_thresh)[0]
                trimmed_cwm = act_contribs[np.min(pass_inds): np.max(pass_inds) + 1]
                trimmed_pwm = pwm[np.min(pass_inds): np.max(pass_inds) + 1]
                
                # Last check to make sure motif is overall positive
                if np.sum(trimmed_cwm) < 0:
                    continue

                cwms.append(trimmed_cwm)
                pwms.append(trimmed_pwm)
                num_seqlets.append(len(seqlets))

    if plot_passed_motifs:
        print("Final motifs: %d total" % len(cwms))
        print("==========================================")
        for i in range(len(cwms)):
            print("Motif %d (%d seqlets)" % (i + 1, num_seqlets[i]))
            viz_sequence.plot_weights(cwms[i])
            viz_sequence.plot_weights(pwms[i])
    return cwms, pwms

In [ ]:
noprior_cwms, noprior_pwms = import_tfmodisco_motifs(noprior_tfm_path, plot_all_motifs=False, plot_passed_motifs=True)

In [ ]:
prior_cwms, prior_pwms = import_tfmodisco_motifs(prior_tfm_path, plot_all_motifs=False, plot_passed_motifs=True)

### Call motif instances

In [ ]:
def jaccard_sim(seq_1, seq_2, normalize=True):
    """
    Takes two windows (W x 4 arrays) and computes a similarity between them,
    using a continuous Jaccard metric. If `normalize` is True, L1 normalizes
    both sequences first.
    """
    if normalize:
        seq_1 = seq_1 / np.sum(np.abs(seq_1))
        seq_2 = seq_2 / np.sum(np.abs(seq_2))
    ab_1, ab_2 = np.abs(seq_1), np.abs(seq_2)
    inter = np.minimum(ab_1, ab_2) * np.sign(seq_1) * np.sign(seq_2)
    union = np.maximum(ab_1, ab_2)
    cont_jaccard = np.sum(inter, axis=1) / np.sum(union, axis=1)
    return np.sum(cont_jaccard)

In [ ]:
def call_motifs(coords, imp_scores, one_hot_seqs, cwms, pwms, top_contrib_frac=0.1):
    """
    From an N x 3 object array of coordinates and parallel N x I x 4 array
    of actual importance scores, this calls motifs based on Jaccard similarity.
    For each motif in `cwms` and `pwms`, scans to find the best instances. Returns
    the result of all scans (over all motifs together) as an M x 3 object array
    of instance coordinates, a parallel M-array of match scores, and an M-array
    of CWM source indices, all in reverse order by match score.
    """
    assert len(coords) == len(imp_scores)
    assert np.all(coords[:, 2] - coords[:, 1] == input_length), \
        "Expect all coordinates to be of length %d" % input_length
    assert imp_scores.shape[1] == input_length, \
        "Expect important scores to have sequence length %d" % input_length
    
    # Allocate arrays
    array_len = 0
    for cwm in cwms:
        num_windows_per_coord = input_length - len(cwm) + 1
        array_len += num_windows_per_coord * len(coords)
    motif_coords = np.empty((array_len, 3), dtype=object)
    cwm_scores = np.empty(array_len)  # CWM match score; for ranking at the end
    pwm_scores = np.empty(array_len)  # PWM match score; for filtering
    motif_sources = np.empty(array_len, dtype=int)
    contrib_scores = np.empty(array_len)  # Total contribution score; for filtering
    
    next_index = 0
    # For each coordinate, for each CWM, for each window, compute the match
    for i in tqdm.notebook.trange(len(coords)):
        chrom, start, end = coords[i]
        imp_score_track = imp_scores[i]
        one_hot_seq_track = one_hot_seqs[i]
            
        for motif_i in range(len(cwms)):
            cwm, pwm = cwms[motif_i], pwms[motif_i]
            rc_cwm, rc_pwm = revcomp_motif(cwm), revcomp_motif(pwm)
            motif_len = len(cwm)
            num_windows_per_coord = input_length - motif_len + 1

            for j in range(num_windows_per_coord):
                motif_coords[next_index, 0] = chrom
                motif_coords[next_index, 1] = start + j
                motif_coords[next_index, 2] = start + j + motif_len
                imp_score_window = imp_score_track[j:(j + motif_len)]
                one_hot_seq_window = one_hot_seq_track[j:(j + motif_len)]
                fwd_score, rev_score = jaccard_sim(imp_score_window, cwm), jaccard_sim(imp_score_window, rc_cwm)
                if fwd_score > rev_score:
                    cwm_scores[next_index] = fwd_score
                    pwm_scores[next_index] = np.sum(one_hot_seq_window * pwm)
                else:
                    cwm_scores[next_index] = rev_score
                    pwm_scores[next_index] = np.sum(one_hot_seq_window * rc_pwm)
                motif_sources[next_index] = motif_i
                contrib_scores[next_index] = np.sum(np.abs(imp_score_window))
                next_index += 1

    # Filter out anything with a negative PWM match score
    mask = pwm_scores > 0
    motif_coords = motif_coords[mask]
    cwm_scores = cwm_scores[mask]
    motif_sources = motif_sources[mask]
    contrib_scores = contrib_scores[mask]
    
    # For each CWM, keep only the top few based on CWM match score
    mask = np.zeros(len(motif_sources), dtype=bool)
    for cwm_i in range(len(cwms)):
        cwm_mask = motif_sources == cwm_i
        cwm_match_scores = cwm_scores[cwm_mask]
        limit = np.flip(np.sort(cwm_match_scores))[int(len(cwm_match_scores) * top_contrib_frac)]
        mask = mask | (cwm_mask & (cwm_scores >= limit))  # Update mask for wherever it is this CWM and score in top %ile
    motif_coords = motif_coords[mask]
    cwm_scores = cwm_scores[mask]
    motif_sources = motif_sources[mask]
    contrib_scores = contrib_scores[mask]
    
    # Now order by contribution score
    inds = np.flip(np.argsort(contrib_scores))
    return motif_coords[inds], cwm_scores[inds], motif_sources[inds]

In [ ]:
noprior_motif_call_coords, noprior_cwm_scores, noprior_motif_sources = call_motifs(
    sample_coords, (noprior_imp_scores * sample_input_seqs), sample_input_seqs, noprior_cwms, noprior_pwms
)
prior_motif_call_coords, prior_cwm_scores, prior_motif_sources = call_motifs(
    sample_coords, (prior_imp_scores * sample_input_seqs), sample_input_seqs, prior_cwms, prior_pwms
)

In [ ]:
# Plot some examples of motif calls
for i in np.random.choice(1000, size=10, replace=False):
    chrom, start, end = prior_motif_call_coords[i]
    sample_index = np.where(
        (sample_coords[:, 0] == chrom) & (sample_coords[:, 1] <= start) & (sample_coords[:, 2] > end)
    )[0][0]
    _, sample_start, sample_end = sample_coords[sample_index]
    seq_start = start - sample_start
    seq_end = seq_start + (end - start)
    viz_sequence.plot_weights(prior_imp_scores[sample_index][seq_start:seq_end] * sample_input_seqs[sample_index][seq_start:seq_end])

### Quantify agreement of motif instances with footprints using ranks

In [ ]:
def compute_footprint_overlap(coords, footprints):
    """
    From an N x 3 object array of coordinates and a table of imported footprints,
    this function determines which coordinates overlap with a footprint.
    Returns a N-array binary mask, denoting which coordinates overlap with a
    footprint.
    """
    mask = np.zeros(len(coords), dtype=int)
    for i in tqdm.notebook.trange(len(coords)):
        chrom, start, end = coords[i]
        fps = footprints[
            (footprints["chrom"] == chrom) & (footprints["peak_start"] <= end) & \
            (start <= footprints["peak_end"])
        ]
        if len(fps):
            mask[i] = 1
    return mask

In [ ]:
noprior_motif_overlap_mask = compute_footprint_overlap(
    noprior_motif_call_coords[:10000], footprints
)
prior_motif_overlap_mask = compute_footprint_overlap(
    prior_motif_call_coords[:10000], footprints
)
noprior_motif_overlap_cdf = normed_rank_enrichment(noprior_motif_overlap_mask)
prior_motif_overlap_cdf = normed_rank_enrichment(prior_motif_overlap_mask)

In [ ]:
np.unique(prior_motif_sources[:1000], return_counts=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
title = "Cumulative BPNet %s ChIP-nexus peak overlap in top motif calls" % ("aggregated" if task_index is None else bpnet_tasks[task_index])
title += "\n%s %s models" % (condition_name, model_type)
title += "\nComputed on %d randomly drawn test peaks" % num_samples
plt.title(title)
ax.plot(np.cumsum(noprior_motif_overlap_mask), label="No prior", color="coral")
ax.plot(np.cumsum(prior_motif_overlap_mask), label="With Fourier prior", color="slateblue")
ax.set_ylabel("Number of motif calls in footprints")
ax.set_xlabel("Top k motif calls by importance")
plt.legend()

### Quantify agreement of motif instances with profile-derived footprints using ranks
Instead of using pre-determined footprints, we use profile heights to binarize footprint classification

In [ ]:
def binarize_footprints_by_profiles(coords, profile_func, flank_size=35, quant_cutoff=0.9):
    """
    Given an N x 3 object array of coordinates and a function that returns
    the profile given any coordinate (chrom, start, and end), determines
    which of the N coordinates overlie a footprint, based on profile counts.
    For each coordinate, this will look `flank_size` base pairs left and right
    of the center, and record the maximum profile height seen. A coordinate is
    deemed to overlie a footprint if its maximum profile height is in the
    `quant_cutoff` percentile (default 90th percentile) over all given
    coordinates. Returns an N-array as a binary mask of which centers overlie a
    footprint.
    """
    heights = np.empty(len(coords))
    for i in tqdm.notebook.trange(len(coords)):
        chrom, start, end = coords[i]
        center = (start + end) // 2
        heights[i] = np.max(
            profile_func(chrom, center - flank_size, center + flank_size)
        )
    inds = np.flip(np.argsort(heights))  # Best (highest) height first
    num_keep = int(quant_cutoff * len(coords))
    mask = np.zeros(len(coords), dtype=int)
    mask[inds[:num_keep]] = 1
    return mask

In [ ]:
noprior_motif_profoverlap_mask = binarize_footprints_by_profiles(
    noprior_motif_call_coords[:10000], get_profiles
)
prior_motif_profoverlap_mask = binarize_footprints_by_profiles(
    prior_motif_call_coords[:10000], get_profiles
)
noprior_motif_profoverlap_cdf = normed_rank_enrichment(noprior_motif_profoverlap_mask)
prior_motif_profoverlap_cdf = normed_rank_enrichment(prior_motif_profoverlap_mask)

In [ ]:
assert condition_name == "BPNet"
plt.figure()
title = "Cumulative BPNet %s ChIP-nexus peak overlap by profile height in top motif calls" % ("aggregated" if task_index is None else bpnet_tasks[task_index])
title += "\n%s %s models" % (condition_name, model_type)
title += "\nOver %d random test peaks" % len(sample)
plt.title(title)
plt.plot(np.cumsum(noprior_motif_profoverlap_mask), label="No prior", color="red")
plt.plot(np.cumsum(prior_motif_profoverlap_mask), label="With Fourier prior", color="blue")
plt.ylabel("Number of overlapping motif calls")
plt.xlabel("Top k motif calls")
plt.legend()